#### 1. Изучите структуру модуля src


Модуль состоит из двух .py файлов.
1. metrics.py включает в себя метрики для оценки качества
2. utils.py с полезными функциями

In [45]:
import sys
PATH_TO_SRC = '../'
sys.path.append(PATH_TO_SRC)

import warnings
warnings.filterwarnings('ignore')

'/mnt/c/Users/Zalut/Documents/GitHub/GBEdu/Recomendation_sisten'

#### 2. Перенесите функции prefilter_items и postfilter_items из вебинара в модуль src.utils.py


In [46]:
pref = """

def prefilter_items(data, data_train):
    # Уберем самые популярные товары (их и так купят)
    popularity = data_train.groupby('item_id')['user_id'].nunique().reset_index() / data_train['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    
    top_popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]
    
    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]
    
    print(data.shape[0])
    
    return data
    
"""

postf = """

def postfilter_items(user_id, recommednations):
    pass

"""

In [47]:
f = open("src\\utils.py","w+", encoding='utf-8')
f.write(pref + postf)
f.close() 

#### 3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.utils.py


In [48]:
item_rec = """
def get_similar_items_recommendation(user, model, item, N=5):
    '''Рекомендуем товары, похожие на топ-N купленных юзером товаров'''
    res = [id_to_itemid[rec[0]] for rec in model.similar_items(itemid_to_id[item], N=N)]
    return res
    
"""


user_rec = """

def get_similar_users_recommendation(user, model, sparse_user_item, item, N=5):
    '''Рекомендуем топ-N товаров'''

    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[item]],  
                                    recalculate_user=True)]
    return res
        
"""

In [49]:
f = open("src\\utils.py","w+", encoding='utf-8')
f.write(pref + postf + item_rec + user_rec)
f.close() 

#### 4. Создайте модуль src.recommenders.py. Напищите код для класса ниже и положите его в src.recommenders.py


In [50]:
rec = '''
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, weighting=True):
        
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
        
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
     
    @staticmethod
    def prepare_matrix(data):
        
        user_item_matrix = pd.pivot_table(data=data, 
                                          index='user_id', columns='item_id', 
                                          values='quantity',
                                          aggfunc='count', 
                                          fill_value=0)                                         
        
        user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit
        
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads)
        model.fit(csr_matrix(self.user_item_matrix).T.tocsr())
        
        return model

    def get_similar_items_recommendation(user, model, item, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        res = [id_to_itemid[rec[0]] for rec in model.similar_items(
            itemid_to_id[item], N
        )]

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
        res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[item]],  
                                    recalculate_user=True)]

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
        
'''

In [51]:
f = open("src\\recommenders.py","w+", encoding='utf-8')
f.write(rec)
f.close() 

#### 5. Проверьте, что все модули корректно импортируются


In [52]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender
